In [1]:
import json
import os
from sedona.spark import *
from pyspark.sql.functions import expr
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter

In [2]:
# Configure SedonaContext, specify credentials for AWS S3 bucket(s) (optional)

config = SedonaContext.builder(). \
    config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"). \
    getOrCreate()

sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
BB_S3_URL = "s3://wherobots-examples/data/examples/birdbuddy_oct23.csv"

In [4]:
bb_df = sedona.read.format('csv').option('header','true').option('delimiter', ',').load(BB_S3_URL)
bb_df.show(5, truncate=False)

+---+-------------------+--------------------+-----------------------+--------------------+----------------------+
|_c0|anonymized_latitude|anonymized_longitude|timestamp              |common_name         |scientific_name       |
+---+-------------------+--------------------+-----------------------+--------------------+----------------------+
|10 |34.393112          |-118.59075          |2023-10-01 00:00:02.415|california scrub jay|aphelocoma californica|
|11 |34.393112          |-118.59075          |2023-10-01 00:00:02.415|california scrub jay|aphelocoma californica|
|26 |34.393112          |-118.59075          |2023-10-01 00:00:04.544|california scrub jay|aphelocoma californica|
|27 |34.393112          |-118.59075          |2023-10-01 00:00:04.544|california scrub jay|aphelocoma californica|
|34 |34.393112          |-118.59075          |2023-10-01 00:00:05.474|california scrub jay|aphelocoma californica|
+---+-------------------+--------------------+-----------------------+----------

In [5]:
bb_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- anonymized_latitude: string (nullable = true)
 |-- anonymized_longitude: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- common_name: string (nullable = true)
 |-- scientific_name: string (nullable = true)



🔹 ST_Point(CAST(anonymized_longitude AS float), CAST(anonymized_latitude AS float)) AS location
Creates a location column of type geographic point (geometry).

anonymized_longitude and anonymized_latitude are cast to float to match the format expected by ST_Point (Sedona's spatial function).

The result is a new location column containing POINT(lon lat) objects.

🔹 CAST(timestamp AS timestamp) AS timestamp
Casts a timestamp column from a text (or other) format to a real timestamp type that Spark understands for timing operations.

In [6]:
bb_df = bb_df.selectExpr('ST_Point(CAST(anonymized_longitude AS float), CAST(anonymized_latitude AS float)) AS location', 'CAST(timestamp AS timestamp) AS timestamp', 'common_name', 'scientific_name')
bb_df.createOrReplaceTempView('bb')
bb_df.show(15, truncate=False)

+---------------------------------------------+-----------------------+--------------------+----------------------+
|location                                     |timestamp              |common_name         |scientific_name       |
+---------------------------------------------+-----------------------+--------------------+----------------------+
|POINT (-118.59075164794922 34.39311218261719)|2023-10-01 00:00:02.415|california scrub jay|aphelocoma californica|
|POINT (-118.59075164794922 34.39311218261719)|2023-10-01 00:00:02.415|california scrub jay|aphelocoma californica|
|POINT (-118.59075164794922 34.39311218261719)|2023-10-01 00:00:04.544|california scrub jay|aphelocoma californica|
|POINT (-118.59075164794922 34.39311218261719)|2023-10-01 00:00:04.544|california scrub jay|aphelocoma californica|
|POINT (-118.59075164794922 34.39311218261719)|2023-10-01 00:00:05.474|california scrub jay|aphelocoma californica|
|POINT (-118.59075164794922 34.39311218261719)|2023-10-01 00:00:05.474|c

In [7]:
bb_df.printSchema()

root
 |-- location: geometry (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- common_name: string (nullable = true)
 |-- scientific_name: string (nullable = true)



In [8]:
bb_df.count()

9502150

In [10]:
S3_NE_ADMIN1_URL = "s3://wherobots-examples/data/ne_10m_admin_1_states_provinces/"

Load spatial data from a shapefile and transform it into a Spark DataFrame ready for SQL queries.
🔹 spatialRDD = ShapefileReader.readToGeometryRDD(sedona, S3_NE_ADMIN1_URL)
Loads a shapefile from location S3_NE_ADMIN1_URL.

Creates a GeometryRDD — a distributed set of geometric data (e.g. administrative boundaries, regions, etc.).

🔹 admin_df = Adapter.toDf(spatialRDD, sedona)
Converts a GeometryRDD to a Spark DataFrame.

Allows further processing of data in Spark SQL style (i.e. queries, filters, joins).

In [11]:
spatialRDD = ShapefileReader.readToGeometryRDD(sedona, S3_NE_ADMIN1_URL)
admin_df = Adapter.toDf(spatialRDD, sedona)
admin_df.createOrReplaceTempView("admins")
admin_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- adm1_code: string (nullable = true)
 |-- diss_me: string (nullable = true)
 |-- iso_3166_2: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- iso_a2: string (nullable = true)
 |-- adm0_sr: string (nullable = true)
 |-- name: string (nullable = true)
 |-- name_alt: string (nullable = true)
 |-- name_local: string (nullable = true)
 |-- type: string (nullable = true)
 |-- type_en: string (nullable = true)
 |-- code_local: string (nullable = true)
 |-- code_hasc: string (nullable = true)
 |-- note: string (nullable = true)
 |-- hasc_maybe: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_cod: string (nullable = true)
 |-- provnum_ne: string (nullable = true)
 |-- gadm_level: string (nullable = true)
 |-- check_me: string (nullable = true)
 |-- datarank: string (nullable = true)
 |-- abbrev: string (nullable = 

In [12]:
selected_df = sedona.sql("""
SELECT geometry, iso_3166_2 
FROM admins
""")

In [13]:
selected_df.show()

+--------------------+----------+
|            geometry|iso_3166_2|
+--------------------+----------+
|POLYGON ((-58.200...|      AR-E|
|POLYGON ((-58.200...|     UY-PA|
|MULTIPOLYGON (((1...|     ID-KI|
|MULTIPOLYGON (((1...|     MY-12|
|POLYGON ((-69.510...|     CL-AP|
|POLYGON ((-69.290...|      BO-L|
|POLYGON ((-68.989...|      BO-O|
|POLYGON ((-68.989...|     CL-TA|
|POLYGON ((-68.597...|      BO-P|
|POLYGON ((-68.531...|     CL-AN|
|POLYGON ((-69.510...|    PE-TAC|
|POLYGON ((-67.284...|      AR-A|
|POLYGON ((-67.251...|      AR-Y|
|POLYGON ((33.7779...|  -99-X16~|
|MULTIPOLYGON (((3...|     CY-03|
|POLYGON ((77.8003...|     IN-LA|
|POLYGON ((77.8831...|     CN-XJ|
|POLYGON ((78.9176...|     CN-XZ|
|POLYGON ((34.2483...|      IL-D|
|POLYGON ((34.4812...|    PS-GZZ|
+--------------------+----------+
only showing top 20 rows



In [14]:
selected_df.count()

4596

In [15]:
bb_df.count()

9502150

In [16]:
selected_df.createOrReplaceTempView("admins")

In [17]:
bb_admin_df = sedona.sql("""
SELECT bb.location AS location, bb.timestamp AS timestamp, bb.common_name AS common_name, bb.scientific_name AS scientific_name, admins.iso_3166_2 AS state 
FROM bb
JOIN admins 
WHERE ST_Intersects(admins.geometry, bb.location)
""").repartition("state")



In [18]:
bb_admin_df.count()

9406992

In [19]:
bb_admin_df.show()

+--------------------+--------------------+--------------------+--------------------+-----+
|            location|           timestamp|         common_name|     scientific_name|state|
+--------------------+--------------------+--------------------+--------------------+-----+
|POINT (-85.338401...|2023-10-06 15:42:...|     tufted titmouse|  baeolophus bicolor|US-TN|
|POINT (-85.338401...|2023-10-06 15:42:...|     tufted titmouse|  baeolophus bicolor|US-TN|
|POINT (-85.338401...|2023-10-06 15:42:...|  carolina chickadee|poecile carolinensis|US-TN|
|POINT (-85.338401...|2023-10-06 15:42:...|  carolina chickadee|poecile carolinensis|US-TN|
|POINT (-85.338401...|2023-10-06 15:43:...|     tufted titmouse|  baeolophus bicolor|US-TN|
|POINT (-85.338401...|2023-10-06 15:43:...|     tufted titmouse|  baeolophus bicolor|US-TN|
|POINT (-85.146102...|2023-10-06 15:43:...|       mourning dove|    zenaida macroura|US-TN|
|POINT (-85.146102...|2023-10-06 15:43:...|       mourning dove|    zenaida macr

In [20]:
bb_admin_df.createOrReplaceTempView("bb_admin")

Counting the number of unique species (common_name) in different states (state). Here's a detailed explanation:

SELECT state, COUNT(DISTINCT common_name) AS unique_species_count
state: Selects the state column, which is the name of the state (or administrative region).

COUNT(DISTINCT common_name) AS unique_species_count: For each state, counts the number of unique species (common_name). The COUNT(DISTINCT ...) function counts the number of unique values in the column, which in this case means counting unique species.

The result of this count will be stored in a new column called unique_species_count.
🔹 FROM bb_admin
🔹 GROUP BY state
Groups the query results by the state column, which means counting unique species for each state separately.
ORDER BY unique_species_count DESC
Sorts the query results in descending order (DESC) by the number of unique species in each state.

In [21]:
unique_species_count_df = sedona.sql("""
SELECT state, COUNT(DISTINCT common_name) AS unique_species_count
FROM bb_admin
GROUP BY state
ORDER BY unique_species_count DESC
""")

unique_species_count_df.show()

+-----+--------------------+
|state|unique_species_count|
+-----+--------------------+
|US-CA|                 110|
|US-TX|                 105|
|US-AZ|                  99|
|US-CO|                  88|
|US-KS|                  78|
|US-WA|                  75|
|US-NM|                  75|
|US-OR|                  74|
|US-FL|                  74|
|US-NY|                  71|
|US-VA|                  69|
|US-NC|                  69|
|US-IL|                  69|
|US-PA|                  68|
|US-GA|                  67|
|US-MN|                  66|
|US-WI|                  65|
|US-MO|                  65|
|US-UT|                  65|
|US-MI|                  64|
+-----+--------------------+
only showing top 20 rows



Count the number of occurrences of each species (common_name) in different states (state).

SELECT state, common_name, COUNT(*) AS species_count:
state: Selects the name of the state in which the species was recorded.
common_name: Selects the species name (common name).
COUNT(*) AS species_count: Counts the number of occurrences of each species in a given state. The COUNT(*) function counts the number of rows in each group (where each group is a combination of a state and a species). The result is assigned to a new column called species_count.

🔹 FROM bb_admin
🔹 GROUP BY state, common_name
A result group is created from two columns: state and common_name.

For each combination of state and species, the COUNT(*) function will count the number of times a species appears in that state.
🔹 ORDER BY state, species_count DESC
Sorts the results:
First, the results will be sorted ascending by state.

Then, for each state, the results will be sorted descending (DESC) by species_count. This means that the species with the highest number of occurrences in a given state will appear first.
🔹 species_count_df.show()
show() will display the query result in a table, showing the count of species occurrences for each state. This will show rows with columns:
state: State name.
common_name: Species name.
species_count: Number of occurrences of a given species in a given state.

In [22]:
species_count_df = sedona.sql("""
SELECT 
    state, 
    common_name, 
    COUNT(*) AS species_count
FROM bb_admin
GROUP BY state, common_name
ORDER BY state, species_count DESC
""")

species_count_df.show()

+------+--------------------+-------------+
| state|         common_name|species_count|
+------+--------------------+-------------+
| AD-02|           great tit|            4|
| AD-08|   eurasian blue tit|          100|
| AD-08|      european robin|           98|
| AD-08|           great tit|           30|
| AD-08|    common blackbird|            4|
| AE-DU|             sparrow|          566|
| AE-DU|       laughing dove|          224|
| AE-DU|              pigeon|          132|
| AE-DU|eurasian collared...|          112|
| AE-DU|  white eared bulbul|           40|
| AE-DU|              thrush|           20|
| AE-DU|              bulbul|           20|
| AE-DU|             sunbird|            6|
| AE-DU|            starling|            4|
| AE-DU|    common blackbird|            2|
| AE-DU|                crow|            2|
| AE-DU|              weaver|            2|
|AO-LUA|              pigeon|         1270|
|AO-LUA|   cordon bleu finch|          750|
|AO-LUA|             sparrow|   

In [23]:
sedona.sql("""
WITH distinct_states AS (SELECT DISTINCT state FROM bb_admin)
SELECT COUNT(*) AS num FROM distinct_states
""").show()

+---+
|num|
+---+
|893|
+---+



In [24]:
sedona.sql("""
WITH distinct_birds AS (SELECT DISTINCT common_name FROM bb_admin)
SELECT COUNT(*) AS num FROM distinct_birds
""").show()

+---+
|num|
+---+
|370|
+---+



Extracting the number of records for each state, without sorting the data. This query will return the number of records assigned to each state in the bb_admin table. Result is number of administrative units for each state.

In [25]:
state_count_query = sedona.sql("""
SELECT COUNT(*) AS num, state
FROM bb_admin
GROUP BY state
""")
state_count_query.show()

+------+------+
|   num| state|
+------+------+
|124202| US-TN|
|    82|SI-140|
|   384| IT-AO|
|   686|GB-ARM|
| 17238|GB-WSM|
|  8534|BE-VLI|
|  1250|SI-043|
|  2842|  ES-M|
|  1182| IT-PG|
|  2094|GB-NAY|
| 33670| US-OK|
|  8224|GB-NYK|
|  1352| AU-NT|
|  2134| FR-86|
|  1228|GB-STE|
|  1014|SI-040|
|  3796| CZ-VY|
|  3230|GB-SOM|
|  5592| NO-11|
|   470| PT-16|
+------+------+
only showing top 20 rows



In [26]:
geometry_query = sedona.sql("""
SELECT iso_3166_2, geometry
FROM admins
""")
geometry_query.show()

+----------+--------------------+
|iso_3166_2|            geometry|
+----------+--------------------+
|      AR-E|POLYGON ((-58.200...|
|     UY-PA|POLYGON ((-58.200...|
|     ID-KI|MULTIPOLYGON (((1...|
|     MY-12|MULTIPOLYGON (((1...|
|     CL-AP|POLYGON ((-69.510...|
|      BO-L|POLYGON ((-69.290...|
|      BO-O|POLYGON ((-68.989...|
|     CL-TA|POLYGON ((-68.989...|
|      BO-P|POLYGON ((-68.597...|
|     CL-AN|POLYGON ((-68.531...|
|    PE-TAC|POLYGON ((-69.510...|
|      AR-A|POLYGON ((-67.284...|
|      AR-Y|POLYGON ((-67.251...|
|  -99-X16~|POLYGON ((33.7779...|
|     CY-03|MULTIPOLYGON (((3...|
|     IN-LA|POLYGON ((77.8003...|
|     CN-XJ|POLYGON ((77.8831...|
|     CN-XZ|POLYGON ((78.9176...|
|      IL-D|POLYGON ((34.2483...|
|    PS-GZZ|POLYGON ((34.4812...|
+----------+--------------------+
only showing top 20 rows



In [27]:
species_count_df.createOrReplaceTempView("species_counts")
geometry_query.createOrReplaceTempView("geometries")


final_query1 = sedona.sql("""
SELECT 
    species_counts.species_count, 
    species_counts.state, 
    species_counts.common_name,
    geometries.geometry
FROM species_counts
JOIN geometries 
ON species_counts.state = geometries.iso_3166_2
""")


final_query1.show()

+-------------+-----+--------------------+--------------------+
|species_count|state|         common_name|            geometry|
+-------------+-----+--------------------+--------------------+
|          586| AT-7|        eurasian jay|MULTIPOLYGON (((1...|
|          340| AT-7|    common blackbird|MULTIPOLYGON (((1...|
|          174| AT-7|   eurasian blue tit|MULTIPOLYGON (((1...|
|         1322| AT-7|           great tit|MULTIPOLYGON (((1...|
|          672| AT-7|    common chaffinch|MULTIPOLYGON (((1...|
|           64| AT-7| european greenfinch|MULTIPOLYGON (((1...|
|           56| AT-7|eurasian tree spa...|MULTIPOLYGON (((1...|
|           34| AT-7|             dunnock|MULTIPOLYGON (((1...|
|           14| AT-7|            coal tit|MULTIPOLYGON (((1...|
|          166| AT-7|      european robin|MULTIPOLYGON (((1...|
|           28| AT-7|   eurasian nuthatch|MULTIPOLYGON (((1...|
|           14| AT-7|           marsh tit|MULTIPOLYGON (((1...|
|            6| AT-7|         common myn

In [1]:
final_query1.createOrReplaceTempView("final")

NameError: name 'final_query1' is not defined

Executing an SQL query to retrieve only data for the first three states

In [29]:
result_df = sedona.sql("""
SELECT * FROM final
WHERE state IN (
    SELECT state FROM final
    GROUP BY state
    ORDER BY state
    LIMIT 3
)
""")

result_df.show()

+-------------+-----+--------------------+--------------------+
|species_count|state|         common_name|            geometry|
+-------------+-----+--------------------+--------------------+
|          100|AD-08|   eurasian blue tit|POLYGON ((1.65698...|
|           30|AD-08|           great tit|POLYGON ((1.65698...|
|           98|AD-08|      european robin|POLYGON ((1.65698...|
|            4|AD-08|    common blackbird|POLYGON ((1.65698...|
|            4|AD-02|           great tit|POLYGON ((1.59734...|
|           40|AE-DU|  white eared bulbul|MULTIPOLYGON (((5...|
|          224|AE-DU|       laughing dove|MULTIPOLYGON (((5...|
|          112|AE-DU|eurasian collared...|MULTIPOLYGON (((5...|
|          566|AE-DU|             sparrow|MULTIPOLYGON (((5...|
|           20|AE-DU|              thrush|MULTIPOLYGON (((5...|
|          132|AE-DU|              pigeon|MULTIPOLYGON (((5...|
|           20|AE-DU|              bulbul|MULTIPOLYGON (((5...|
|            2|AE-DU|                cro

In [31]:
result_df.createOrReplaceTempView("final1")

states = [row["state"] for row in sedona.sql("SELECT DISTINCT state FROM final1").collect()]
sedona.sql("SELECT DISTINCT state FROM final1"):

This is an SQL query that selects all unique (DISTINCT) values ​​from the state column in the final1 table.

.collect():

The collect() function executes an SQL query on the dataset and returns a result in the form of a list of rows. For each row in the result, row["state"] allows you to get the value from the state column.

[row["state"] for row in ...]:

Creates a list where each element is the value from the state column for each row in the query results.

As a result, the states variable contains a list of unique states (e.g. ["Texas", "California", "New York"]).

2. state_columns = ", ".join([f"'{state}'" for state in states])
[f"'{state}'" for state in states]:

Creates a list where each state in the list states is wrapped in single quotes. This creates a list where each state is stored as a SQL string (e.g. "'Texas'").

For example, if states = ["Texas", "California", "New York"], the result of this operation is: ["'Texas'", "'California'", "'New York'"].

", ".join([...]):

Joins all the elements of the list into a single string, where each element is separated by a comma and a space.

In the case where the list is ["'Texas'", "'California'", "'New York'"], after this operation, the result of this operation is:

In [32]:
states = [row["state"] for row in sedona.sql("SELECT DISTINCT state FROM final1").collect()]

state_columns = ", ".join([f"'{state}'" for state in states])

In [33]:
states

['AD-08', 'AD-02', 'AE-DU']

In [34]:
state_columns

"'AD-08', 'AD-02', 'AE-DU'"

In [35]:
query = f"""
SELECT * FROM (
    SELECT 
        common_name, 
        state, 
        species_count
    FROM final1
) 
PIVOT (
    SUM(species_count) FOR state IN ({state_columns})
)
"""

pivot_species_count_df = sedona.sql(query)

In [36]:
pivot_species_count_df.show(3)

+-----------------+-----+-----+-----+
|      common_name|AD-08|AD-02|AE-DU|
+-----------------+-----+-----+-----+
| common blackbird|    4| NULL|    2|
|eurasian blue tit|  100| NULL| NULL|
|   european robin|   98| NULL| NULL|
+-----------------+-----+-----+-----+
only showing top 3 rows



In [37]:
states = [row["state"] for row in sedona.sql("SELECT DISTINCT state FROM final").collect()]
state_columns = ", ".join([f"'{state}'" for state in states])

In [38]:
query = f"""
SELECT * FROM (
    SELECT 
        common_name, 
        state, 
        species_count
    FROM final
) 
PIVOT (
    SUM(species_count) FOR state IN ({state_columns})
)
"""

# Wykonanie zapytania
pivot_species_count_df = sedona.sql(query)

In [39]:

state_count_query.createOrReplaceTempView("state_counts")
geometry_query.createOrReplaceTempView("geometries")


final_query = sedona.sql("""
SELECT 
    state_counts.num, 
    state_counts.state, 
    geometries.geometry
FROM state_counts
JOIN geometries 
ON state_counts.state = geometries.iso_3166_2
""")

# Wyświetlenie wyniku
final_query.show()

+----+------+--------------------+
| num| state|            geometry|
+----+------+--------------------+
|1352| AU-NT|MULTIPOLYGON (((1...|
|2854| AU-SA|MULTIPOLYGON (((1...|
|8534|BE-VLI|MULTIPOLYGON (((5...|
|6258| CA-NL|MULTIPOLYGON (((-...|
|8548| CZ-JM|POLYGON ((15.7037...|
|5594| CZ-PL|POLYGON ((12.4825...|
|3796| CZ-VY|POLYGON ((16.3919...|
|3678| DK-81|MULTIPOLYGON (((9...|
|2842|  ES-M|MULTIPOLYGON (((-...|
|  20| ES-SE|POLYGON ((-6.2316...|
| 334| FI-08|POLYGON ((26.1377...|
|1356| FR-17|MULTIPOLYGON (((-...|
| 548| FR-18|POLYGON ((1.88572...|
|1334| FR-22|POLYGON ((-3.6385...|
| 112| FR-26|POLYGON ((4.99006...|
|3480| FR-28|POLYGON ((1.21179...|
|3770| FR-29|MULTIPOLYGON (((-...|
|1704| FR-54|POLYGON ((5.74634...|
|4132| FR-69|POLYGON ((4.79782...|
| 564| FR-70|POLYGON ((5.61595...|
+----+------+--------------------+
only showing top 20 rows



In [40]:
state_count_df = sedona.sql("""
WITH states_count1 AS (SELECT COUNT(*) AS num, state
FROM bb_admin
GROUP BY state
ORDER BY num DESC)
SELECT states_count1.num, states_count1.state, admins.geometry FROM states_count1
JOIN admins ON states_count1.state = admins.iso_3166_2
""")

In [41]:
state_count_df.show()

+----+------+--------------------+
| num| state|            geometry|
+----+------+--------------------+
|1352| AU-NT|MULTIPOLYGON (((1...|
|2854| AU-SA|MULTIPOLYGON (((1...|
|8534|BE-VLI|MULTIPOLYGON (((5...|
|6258| CA-NL|MULTIPOLYGON (((-...|
|8548| CZ-JM|POLYGON ((15.7037...|
|5594| CZ-PL|POLYGON ((12.4825...|
|3796| CZ-VY|POLYGON ((16.3919...|
|3678| DK-81|MULTIPOLYGON (((9...|
|2842|  ES-M|MULTIPOLYGON (((-...|
|  20| ES-SE|POLYGON ((-6.2316...|
| 334| FI-08|POLYGON ((26.1377...|
|1356| FR-17|MULTIPOLYGON (((-...|
| 548| FR-18|POLYGON ((1.88572...|
|1334| FR-22|POLYGON ((-3.6385...|
| 112| FR-26|POLYGON ((4.99006...|
|3480| FR-28|POLYGON ((1.21179...|
|3770| FR-29|MULTIPOLYGON (((-...|
|1704| FR-54|POLYGON ((5.74634...|
|4132| FR-69|POLYGON ((4.79782...|
| 564| FR-70|POLYGON ((5.61595...|
+----+------+--------------------+
only showing top 20 rows



In [42]:
SedonaKepler.create_map(final_query, name="Bird observations by state")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Bird observations by state': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, …

25/04/24 15:28:52 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
